# <center>**Analyse exploratoire des donnees**

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import plotly_express as px

import polars as pl

In [3]:
# Définir les informations de connexion à la base de données
db_user = 'postgres'
db_password = 'HairiaMonAmour_2018' # renseigner votre mot de passe
db_host = 'localhost'  # ou l'adresse IP du serveur de la base de données
db_port = '5432'  # Port par défaut pour PostgreSQL
db_name = 'nyc_yellow_taxi_record'

connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create database engine
engine = create_engine(connection_string)

In [4]:
df_zones = pl.read_database("SELECT * FROM taxi_zone;", engine)
df_zones


locationid,borough,zone,service_zone
i64,str,str,str
1,"""EWR""","""Newark Airport""","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,"""Bronx""","""Allerton/Pelham Gardens""","""Boro Zone"""
4,"""Manhattan""","""Alphabet City""","""Yellow Zone"""
5,"""Staten Island""","""Arden Heights""","""Boro Zone"""
…,…,…,…
261,"""Manhattan""","""World Trade Center""","""Yellow Zone"""
262,"""Manhattan""","""Yorkville East""","""Yellow Zone"""
263,"""Manhattan""","""Yorkville West""","""Yellow Zone"""


In [5]:
type(df_zones)

polars.dataframe.frame.DataFrame

In [5]:
# Extraction des données des trajets en taxi de 2023
query = "SELECT * FROM ytaxi_histo WHERE (trip_distance > 0 AND passenger_count > 0 AND total_amount >= 0) AND (EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023);"

# Définir la taille des morceaux pour la fusion
chunksize = 100000  # par exemple, fusionner par paquets de 100 milles lignes

result_generator = pl.read_database(query, engine, iter_batches=True, batch_size=chunksize)

type(result_generator)

generator

In [6]:
# Création du dossier pour sauvegarder les fichiers Parquet si nécessaire
import os
if not os.path.exists("merged_chunk_parquet"):
    os.makedirs("merged_chunk_parquet")

In [7]:
# Sélectionner uniquement les colonnes pertinentes de df_zones pour réduire la mémoire utilisée
df_zones_subset = df_zones.select([
        "locationid", "borough", "zone", "service_zone"
    ])

# Initialiser une liste pour stocker les morceaux fusionnés
merged_chunks = []

# Parcourir les morceaux de données
#for chunk in result_generator:
for i, chunk in enumerate(result_generator):

    # Fusionner les données des trajets avec les informations sur les zones pour les points de départ
    merged_chunk = chunk.join(
        df_zones_subset.rename({'borough': 'puborough', 'zone': 'puzone', 'service_zone': 'pu_service_zone'}), 
        how='left', left_on='pulocationid', right_on='locationid'
    ) 

    # Fusionner les données des trajets avec les informations sur les zones pour les points de destination
    merged_chunk = merged_chunk.join(
        df_zones_subset.rename({'borough': 'doborough', 'zone': 'dozone', 'service_zone': 'do_service_zone'}), 
        how='left', left_on='dolocationid', right_on='locationid'
    )

    # Créer une nouvelle colonne par concaténation de puborough et doborough
    merged_chunk = merged_chunk.with_columns(
        pl.concat_str(
            merged_chunk['puborough'], pl.lit(' - '), merged_chunk['doborough']
        ).alias("Itineraire Arrondissement")
    )

    # Créer une nouvelle colonne par concaténation de puzone et dozone
    merged_chunk = merged_chunk.with_columns(
        pl.concat_str(
            merged_chunk['puzone'], pl.lit(' - '), merged_chunk['dozone']
        ).alias("Itineraire zone")
    )

    merged_chunk = merged_chunk.drop_nulls()

    # Enregistrer le morceau fusionné au format Parquet
    merged_chunk.write_parquet(f"merged_chunk_parquet/chunk_{i}.parquet")

    # Ajouter le morceau fusionné à la liste
    #merged_chunks.append(merged_chunk)

# Concaténer tous les morceaux fusionnés pour former la DataFrame finale
#df_trips = pl.concat(merged_chunks)


# Supprimer les lignes avec des valeurs manquantes dans les données
#df_trips = df_trips.drop_nulls()

# Enregistrer la DataFrame nettoyée au format Parquet
#df_trips.write_parquet("trips_cleaned.parquet")

In [8]:
from pyspark.sql import SparkSession
import os

# Initialiser SparkSession
spark = SparkSession.builder \
    .appName("Merge Parquet Files") \
    .getOrCreate()

# Chemin du dossier contenant les fichiers Parquet
input_path = "merged_chunk_parquet"

# Chemin du fichier Parquet de sortie
output_path = "merged_chunks_combined.parquet"

# Charger tous les fichiers Parquet dans un DataFrame Spark
df = spark.read.parquet(input_path)

# Écrire le DataFrame combiné au format Parquet
df.write.parquet(output_path)

# Arrêter SparkSession
spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/11 02:06:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/11 02:06:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/05/11 02:07:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/05/11 02:07:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/05/11 02:07:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/05/11 02:07:00 WARN MemoryManager: Total allocation exceeds 95.00% (1

In [9]:
from pyspark.sql import SparkSession

# Initialiser SparkSession
spark = SparkSession.builder \
    .appName("Count Rows in Parquet File") \
    .getOrCreate()

# Chemin du fichier Parquet
parquet_file_path = "merged_chunks_combined.parquet"

# Charger le fichier Parquet dans un DataFrame Spark
df_trips = spark.read.parquet(parquet_file_path)

# Compter le nombre de lignes dans le DataFrame
row_count = df_trips.count()

# Afficher le nombre de lignes
print("Nombre de lignes dans le fichier Parquet:", row_count)

Nombre de lignes dans le fichier Parquet: 35083378


In [10]:
df_trips.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+--------------------+---------------+---------+--------------------+---------------+-------------------------+--------------------+
|vendorid|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|store_and_fwd_flag|pulocationid|dolocationid|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|puborough|              puzone|pu_service_zone|doborough|              dozone|do_service_zone|Itineraire Arrondissement|     Itineraire zone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+------

In [11]:
df_trips.printSchema()

root
 |-- vendorid: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- ratecodeid: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pulocationid: long (nullable = true)
 |-- dolocationid: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- puborough: string (nullable = true)
 |-- puzone: string (nullable = true)
 |-- pu_service_zone: string (nullable = true)
 |-- 

In [ ]:
# Charger uniquement les colonnes pertinentes de df_zones pour réduire la mémoire utilisée
df_zones_subset = df_zones[['locationid', 'borough', 'zone', 'service_zone']]

# Initialiser une liste pour stocker les morceaux fusionnés
merged_chunks = []

# Parcourir les morceaux de df_trips
for chunk in result_generator:
    # Fusionner le morceau actuel avec df_zones_subset sur pulocationid
    merged_chunk = pd.merge(chunk, df_zones_subset.rename(columns={'borough': 'puborough', 'zone': 'puzone', 'service_zone': 'pu_service_zone'}),
                            how='left', left_on='pulocationid', right_on='locationid')
    
    # Fusionner le morceau actuel avec df_zones_subset sur dolocationid
    merged_chunk = pd.merge(merged_chunk, df_zones_subset.rename(columns={'borough': 'doborough', 'zone': 'dozone', 'service_zone': 'do_service_zone'}),
                            how='left', left_on='dolocationid', right_on='locationid')
    
    # Créer une nouvelle colonne par concaténation de puborough et doborough pour chaque morceau
    merged_chunk['Itineraire Arrondissement'] = merged_chunk.apply(lambda row: f"{row['puborough']} - {row['doborough']}", axis=1)

    # Créer une nouvelle colonne par concaténation de puzone et dozone pour chaque morceau
    merged_chunk['Itineraire zone'] = merged_chunk.apply(lambda row: f"{row['puzone']} - {row['dozone']}", axis=1)
    
    # Ajouter le morceau fusionné à la liste
    merged_chunks.append(merged_chunk)

# Concaténer tous les morceaux fusionnés pour former la DataFrame finale
df_trips = pd.concat(merged_chunks, ignore_index=True)

# Supprimer les colonnes redondantes
df_trips.drop(columns=['locationid_x', 'locationid_y'], inplace=True)

# Supprimer les lignes avec des valeurs manquantes dans les donnees
df_trips = df_trips.dropna(axis=0)

# Enregistrer la DataFrame nettoyée au format Parquet
df_trips.to_parquet('trips_cleaned.parquet', index=False)

# Dimension de df_trips
print(df_trips.shape)

# Afficher la DataFrame avec les nouvelles colonnes
df_trips.head()

In [ ]:
import polars as pl

# URL JDBC pour PostgreSQL
jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}"

# Requête SQL pour extraire les données des trajets en taxi de 2023
query = """
    SELECT * FROM ytaxi_histo
    WHERE 
	(trip_distance > 0 AND passenger_count > 0 AND total_amount >= 0) 
	AND 
	(EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023);
"""

# Charger les données avec la requête SQL spécifiée
df_trips = pl.read_sql(jdbc_url, query, properties=connection_properties)

# Charger les données des zones de taxi
df_zones = pl.read_sql(jdbc_url, "taxi_zone", properties=connection_properties)

# Sélectionner uniquement les colonnes pertinentes de df_zones pour réduire la mémoire utilisée
df_zones_subset = df_zones.select([
    "locationid", "borough", "zone", "service_zone"
]).alias(["locationid", "puborough", "puzone", "pu_service_zone"])

# Fusionner les données des trajets avec les informations sur les zones pour les points de départ
df_trips = df_trips.join(df_zones_subset, on="pulocationid", how="left") \
                   .drop_in_place(["locationid"])

# Fusionner les données des trajets avec les informations sur les zones pour les points de destination
df_trips = df_trips.join(df_zones_subset, on="dolocationid", how="left") \
                   .drop_in_place(["locationid"])

# Créer une nouvelle colonne par concaténation de puborough et doborough
df_trips = df_trips.with_column(
    pl.concat_str(
        df_trips['puborough'], pl.lit(' - '), df_trips['doborough']
    ).alias("Itineraire Arrondissement")
)

# Créer une nouvelle colonne par concaténation de puzone et dozone
df_trips = df_trips.with_column(
    pl.concat_str(
        df_trips['puzone'], pl.lit(' - '), df_trips['dozone']
    ).alias("Itineraire zone")
)

# Supprimer les colonnes redondantes
df_trips = df_trips.drop_in_place(["locationid_x", "locationid_y"])

# Supprimer les lignes avec des valeurs manquantes dans les données
df_trips = df_trips.drop_na()

# Enregistrer la DataFrame nettoyée au format Parquet
df_trips.write_parquet("trips_cleaned.parquet")

# Dimension de df_trips
print(df_trips.shape())

# Afficher la DataFrame avec les nouvelles colonnes
df_trips.head()


In [ ]:
# Supprimer les lignes avec des valeurs manquantes dans les donnees
df.select(pl.col("a").drop_nans())
df_trips = df_trips.drop_nans()

# Enregistrer la DataFrame nettoyée au format Parquet
df_trips.to_parquet('trips_cleaned.parquet', index=False)

# Dimension de df_trips
print(df_trips.shape)

# Afficher la DataFrame avec les nouvelles colonnes
df_trips.head()

In [3]:
# Extraction des données des trajets en taxi de 2023
query = """
    SELECT * FROM ytaxi_histo
    WHERE 
	(trip_distance > 0 AND passenger_count > 0 AND total_amount >= 0) 
	AND 
	(EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023);
"""

# Définir la taille des morceaux pour la fusion
chunksize = 100000  # par exemple, fusionner par paquets de 100 milles lignes

result_generator = pd.read_sql(query, engine, chunksize=chunksize)

df_zones = pd.read_sql("SELECT * FROM taxi_zone;", engine)

# Charger uniquement les colonnes pertinentes de df_zones pour réduire la mémoire utilisée
df_zones_subset = df_zones[['locationid', 'borough', 'zone', 'service_zone']]

# Initialiser une liste pour stocker les morceaux fusionnés
merged_chunks = []

# Parcourir les morceaux de df_trips
for chunk in result_generator:
    # Fusionner le morceau actuel avec df_zones_subset sur pulocationid
    merged_chunk = pd.merge(chunk, df_zones_subset.rename(columns={'borough': 'puborough', 'zone': 'puzone', 'service_zone': 'pu_service_zone'}),
                            how='left', left_on='pulocationid', right_on='locationid')
    
    # Fusionner le morceau actuel avec df_zones_subset sur dolocationid
    merged_chunk = pd.merge(merged_chunk, df_zones_subset.rename(columns={'borough': 'doborough', 'zone': 'dozone', 'service_zone': 'do_service_zone'}),
                            how='left', left_on='dolocationid', right_on='locationid')
    
    # Créer une nouvelle colonne par concaténation de puborough et doborough pour chaque morceau
    merged_chunk['Itineraire Arrondissement'] = merged_chunk.apply(lambda row: f"{row['puborough']} - {row['doborough']}", axis=1)

    # Créer une nouvelle colonne par concaténation de puzone et dozone pour chaque morceau
    merged_chunk['Itineraire zone'] = merged_chunk.apply(lambda row: f"{row['puzone']} - {row['dozone']}", axis=1)
    
    # Ajouter le morceau fusionné à la liste
    merged_chunks.append(merged_chunk)

# Concaténer tous les morceaux fusionnés pour former la DataFrame finale
df_trips = pd.concat(merged_chunks, ignore_index=True)

# Supprimer les colonnes redondantes
df_trips.drop(columns=['locationid_x', 'locationid_y'], inplace=True)

# Supprimer les lignes avec des valeurs manquantes dans les donnees
df_trips = df_trips.dropna(axis=0)

# Enregistrer la DataFrame nettoyée au format Parquet
df_trips.to_parquet('trips_cleaned.parquet', index=False)

# Dimension de df_trips
print(df_trips.shape)

# Afficher la DataFrame avec les nouvelles colonnes
df_trips.head()

: 

## Analyse des tentances temporelles des trajets en taxi au fil des mois, des jours de la semaine et des heures

Toutes les analyses seront faites sur l'annee complete la plus recente soit 2023.

In [3]:
# Extraction des données des trajets en taxi
query = """
    SELECT tpep_pickup_datetime, COUNT(*) AS num_trips
    FROM ytaxi_histo
    WHERE 
	(trip_distance > 0 AND passenger_count > 0) 
	AND 
	(EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023)
    AND total_amount > 0
    GROUP BY tpep_pickup_datetime;
"""
df = pd.read_sql(query, engine)
df.head()

,tpep_pickup_datetime,num_trips
0,2023-01-01 00:00:05,1
1,2023-01-01 00:00:06,1
2,2023-01-01 00:00:08,1
3,2023-01-01 00:00:09,1
4,2023-01-01 00:00:13,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18843134 entries, 0 to 18843133
Data columns (total 2 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   num_trips             int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 287.5 MB


In [6]:
# Analyse des tendances temporelles au fil des mois
fig_monthly = px.line(df.resample('ME', on='tpep_pickup_datetime').sum(), 
                      x=df.resample('ME', on='tpep_pickup_datetime').sum().index, 
                      y='num_trips', 
                      title='Tendances mensuelles des trajets en taxi')
fig_monthly.update_xaxes(title='Mois')
fig_monthly.update_yaxes(title='Nombre total de trajets')

# Analyse des tendances temporelles au fil des jours de la semaine
fig_weekly = px.line(df.resample('D', on='tpep_pickup_datetime').sum(), 
                     x=df.resample('D', on='tpep_pickup_datetime').sum().index, 
                     y='num_trips', 
                     title='Tendances journalières des trajets en taxi')
fig_weekly.update_xaxes(title='Jour de la semaine')
fig_weekly.update_yaxes(title='Nombre total de trajets')

# Analyse des tendances temporelles au fil des heures de la journée
fig_hourly = px.line(df.resample('h', on='tpep_pickup_datetime').sum(), 
                     x=df.resample('h', on='tpep_pickup_datetime').sum().index, 
                     y='num_trips', 
                     title='Tendances horaires des trajets en taxi')
fig_hourly.update_xaxes(title='Heure de la journée')
fig_hourly.update_yaxes(title='Nombre total de trajets')

# Affichage des résultats
fig_monthly.show()
fig_weekly.show()
fig_hourly.show()


In [7]:
# Extraire le mois à partir de tpep_pickup_datetime
df['month'] = df['tpep_pickup_datetime'].dt.month

# Agréger les données en fonction du mois
df_monthly = df.groupby('month')['num_trips'].sum().reset_index()

# Afficher la DataFrame résultante
df_monthly

,month,num_trips
0,1,2906641
1,2,2754749
2,3,3217068
3,4,3104611
4,5,3310158
5,6,3112723
6,7,2740824
7,8,2657120
8,9,2631427
9,10,3277334


In [8]:
# Mapper les numéros de mois aux noms des mois
month_names = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}

df_monthly['Month'] = df_monthly['month'].map(month_names)

# Tracez le graphique de barres
fig = px.bar(df_monthly, x="Month", y="num_trips", labels={"Month": "Month", "Num_Trips": "Average Number of Trips"})
fig.update_layout(title="Total of Taxi Trips per Month")
fig.show()

In [9]:
# Extraire le jour de la semaine à partir de tpep_pickup_datetime (0 pour lundi, 1 pour mardi, ..., 6 pour dimanche)
df['day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek

# Agréger les données en fonction du jour de la semaine
df_weekly = df.groupby('day_of_week')['num_trips'].sum().reset_index()

# Afficher la DataFrame résultante
df_weekly

,day_of_week,num_trips
0,0,4491343
1,1,5222006
2,2,5528614
3,3,5641708
4,4,5334559
5,5,5220421
6,6,4512257


In [10]:
# Mapper les numéros de jour de la semaine aux noms des jours de la semaine
day_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}

df_weekly['Day_of_week'] = df_weekly['day_of_week'].map(day_names)

# Tracez le graphique de barres
fig = px.bar(df_weekly, x="Day_of_week", y="num_trips", labels={"Day_of_week": "Day of Week", "Num_Trips": "Average Number of Trips"})
fig.update_layout(title="Total Number of Taxi Trips per Day of Week")
fig.show()

In [14]:
# Créez une fonction pour vérifier si le jour de la semaine est un week-end
def is_weekend(day):
    return day in [5, 6]  # 5 correspond à samedi et 6 correspond à dimanche

# Appliquez la fonction à la colonne "Day_of_week" pour créer la nouvelle colonne booléenne "is_weekend"
df_weekly['is_weekend'] = df_weekly['day_of_week'].apply(lambda x: is_weekend(x))

# Remplacer les valeurs booléennes par des étiquettes explicites
df_weekly['is_weekend'] = df_weekly['is_weekend'].replace({True: 'Weekend', False: 'Week'})

# Affichez le DataFrame avec la nouvelle colonne
df_weekly

,day_of_week,num_trips,Day_of_week,is_weekend
0,0,4491343,Monday,Week
1,1,5222006,Tuesday,Week
2,2,5528614,Wednesday,Week
3,3,5641708,Thursday,Week
4,4,5334559,Friday,Week
5,5,5220421,Saturday,Weekend
6,6,4512257,Sunday,Weekend


In [15]:
# Groupez les données selon la colonne "is_weekend" et faites la somme de "num_trips"
df_grouped = df_weekly.groupby('is_weekend')['num_trips'].sum().reset_index()

# Tracez le Pie Chart
fig = px.pie(df_grouped, values='num_trips', names='is_weekend', title='Percentage of Taxi Trips on Weekends')
fig.show()

In [16]:
# Extraire l'heure à partir de tpep_pickup_datetime
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

# Agréger les données en fonction de l'heure de la journée
df_hourly = df.groupby('pickup_hour')['num_trips'].sum().reset_index()

# Afficher la DataFrame résultante
df_hourly

,pickup_hour,num_trips
0,0,1009915
1,1,676610
2,2,444282
3,3,289555
4,4,185729
5,5,198122
6,6,480997
7,7,954160
8,8,1331709
9,9,1527516


In [17]:
# Tracez le graphique de barres
fig = px.bar(df_hourly, x="pickup_hour", y="num_trips")
fig.update_layout(title="Average Number of Taxi Trips per Hour of the day")
fig.show()

## Analyse des zones

In [4]:
# Extraction des données des trajets en taxi de l'annee 2023
query = """
    SELECT *
    FROM ytaxi_histo
    WHERE 
	(trip_distance > 0 AND passenger_count > 0) 
	AND 
	(EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023);
"""
result_generator = pd.read_sql(query, engine, chunksize=100000)
type(result_generator)

generator

In [5]:
# Initialise une liste pour stocker les chunks de DataFrame
df_chunks = []

# Parcourir le générateur pour obtenir les DataFrame chunk par chunk
for df_chunk in result_generator:
    df_chunks.append(df_chunk)

# Concaténer tous les chunks de DataFrame en une seule DataFrame
df_trips = pd.concat(df_chunks, ignore_index=True)

# Afficher la DataFrame complète
print(df_trips.shape)
df_trips.head()

(35950908, 19)


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-08-02 14:58:35,2023-08-02 15:39:09,1.0,14.59,1.0,N,132,226,2,58.3,0.0,0.5,0.00,0.00,1.0,61.55,0.0,1.75
1,2,2023-08-01 14:35:26,2023-08-01 14:55:15,1.0,2.76,1.0,N,137,262,1,18.4,0.0,0.5,2.24,0.00,1.0,24.64,2.5,0.00
2,1,2023-08-01 14:47:04,2023-08-01 15:15:28,1.0,3.50,1.0,N,231,48,1,23.3,2.5,0.5,5.45,0.00,1.0,32.75,2.5,0.00
3,2,2023-08-01 14:27:50,2023-08-01 14:47:58,1.0,3.08,1.0,N,107,140,2,19.8,0.0,0.5,0.00,0.00,1.0,23.80,2.5,0.00
4,2,2023-08-01 14:55:52,2023-08-01 15:23:06,1.0,9.98,1.0,N,140,138,1,40.1,5.0,0.5,11.13,6.55,1.0,66.78,2.5,0.00


In [6]:
df_zones = pd.read_sql("SELECT * FROM taxi_zone;", engine)
df_zones

,locationid,borough,zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,None,None


In [4]:
# Charger uniquement les colonnes pertinentes de df_zones pour réduire la mémoire utilisée
df_zones_subset = df_zones[['locationid', 'borough', 'zone', 'service_zone']]

# Extraction des données des trajets en taxi de l'annee 2023
query = """
    SELECT *
    FROM ytaxi_histo
    WHERE 
	(trip_distance > 0 AND passenger_count > 0) 
	AND 
	(EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023);
"""

# Définir la taille des morceaux pour la fusion
chunksize = 100000  # par exemple, fusionner par paquets de 100 milles lignes

# Generateur des morceaux de la table ytaxi_histo
result_generator = pd.read_sql(query, engine, chunksize=chunksize)

# Initialiser une liste pour stocker les morceaux fusionnés
merged_chunks = []

# Parcourir les morceaux de df_trips
for chunk in result_generator:
    # Fusionner le morceau actuel avec df_zones_subset sur pulocationid
    merged_chunk = pd.merge(chunk, df_zones_subset.rename(columns={'borough': 'puborough', 'zone': 'puzone', 'service_zone': 'pu_service_zone'}),
                            how='left', left_on='pulocationid', right_on='locationid')
    
    # Fusionner le morceau actuel avec df_zones_subset sur dolocationid
    merged_chunk = pd.merge(merged_chunk, df_zones_subset.rename(columns={'borough': 'doborough', 'zone': 'dozone', 'service_zone': 'do_service_zone'}),
                            how='left', left_on='dolocationid', right_on='locationid')
    
    # Ajouter le morceau fusionné à la liste
    merged_chunks.append(merged_chunk)

# Concaténer tous les morceaux fusionnés pour former la DataFrame finale
df_trips = pd.concat(merged_chunks, ignore_index=True)

# Supprimer les colonnes redondantes
df_trips.drop(columns=['locationid_x', 'locationid_y'], inplace=True)

# Afficher la DataFrame avec les nouvelles colonnes
df_trips.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,...,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,puborough,puzone,pu_service_zone,doborough,dozone,do_service_zone
0,2,2023-08-01 14:35:26,2023-08-01 14:55:15,1.0,2.76,1.0,N,137,262,1,...,1.0,24.64,2.5,0.00,Manhattan,Kips Bay,Yellow Zone,Manhattan,Yorkville East,Yellow Zone
1,1,2023-08-01 14:47:04,2023-08-01 15:15:28,1.0,3.50,1.0,N,231,48,1,...,1.0,32.75,2.5,0.00,Manhattan,TriBeCa/Civic Center,Yellow Zone,Manhattan,Clinton East,Yellow Zone
2,2,2023-08-01 14:27:50,2023-08-01 14:47:58,1.0,3.08,1.0,N,107,140,2,...,1.0,23.80,2.5,0.00,Manhattan,Gramercy,Yellow Zone,Manhattan,Lenox Hill East,Yellow Zone
3,2,2023-08-01 14:55:52,2023-08-01 15:23:06,1.0,9.98,1.0,N,140,138,1,...,1.0,66.78,2.5,0.00,Manhattan,Lenox Hill East,Yellow Zone,Queens,LaGuardia Airport,Airports
4,1,2023-08-01 14:37:36,2023-08-01 15:43:07,2.0,10.10,1.0,N,138,79,2,...,1.0,72.55,2.5,1.75,Queens,LaGuardia Airport,Airports,Manhattan,East Village,Yellow Zone


In [5]:
df_trips.isna().sum()

vendorid                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count               0
trip_distance                 0
ratecodeid                    0
store_and_fwd_flag            0
pulocationid                  0
dolocationid                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
airport_fee                   0
puborough                  6778
puzone                   317540
pu_service_zone          324318
doborough                144915
dozone                   325987
do_service_zone          470902
dtype: int64

Ce résultat de `df_trips.isna().sum()` indique le nombre de valeurs manquantes (NaN) pour chaque colonne de la DataFrame `df_trips`. Voici ce que chaque colonne représente :

- `puborough`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de prise en charge (`pulocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour le nom du borough.
- `puzone`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de prise en charge (`pulocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour le nom de la zone.
- `pu_service_zone`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de prise en charge (`pulocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour la zone de service.
- `doborough`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de dépose (`dolocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour le nom du borough.
- `dozone`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de dépose (`dolocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour le nom de la zone.
- `do_service_zone`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de dépose (`dolocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour la zone de service.

En résumé, les valeurs manquantes dans ces colonnes indiquent des incohérences ou des données manquantes dans la correspondance entre les identifiants de localisation (`pulocationid` et `dolocationid`) dans la DataFrame `df_trips` et les données de la DataFrame `df_zones`. Il peut être nécessaire d'examiner de plus près ces données manquantes pour comprendre pourquoi elles se produisent et comment les traiter en conséquence.

In [6]:
# Supprimer les lignes avec des valeurs manquantes dans les donnees
df_trips = df_trips.dropna(axis=0)

# Enregistrer la DataFrame nettoyée au format Parquet
df_trips.to_parquet('trips_cleaned.parquet', index=False)

In [7]:
print(df_trips.shape)
df_trips.isna().sum()

(35412547, 25)


vendorid                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
ratecodeid               0
store_and_fwd_flag       0
pulocationid             0
dolocationid             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
puborough                0
puzone                   0
pu_service_zone          0
doborough                0
dozone                   0
do_service_zone          0
dtype: int64

### Arrondissements de prise en charge les plus populaires

In [12]:
# Compter le nombre de trajets par arrondissement de prise en charge
pickup_counts = df_trips['puborough'].value_counts().reset_index()
pickup_counts.columns = ['Arrondissement', 'Nombre de trajets']
pickup_counts

,Arrondissement,Nombre de trajets
0,Manhattan,31721709
1,Queens,3456194
2,Brooklyn,184051
3,Bronx,47932
4,Staten Island,1749
5,EWR,912


In [14]:
# Créer le graphique des arrondissements de prise en charge
fig = px.bar(pickup_counts, x='Arrondissement', y='Nombre de trajets', 
             title='Arrondissements de prise en charge',
             labels={'Arrondissement': 'Arrondissement de prise en charge', 'Nombre de trajets': 'Nombre de trajets'})

# Personnaliser la mise en page
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10))

# Afficher le graphique
fig.show()

### Arrondissements de depose les plus populaires

In [15]:
# Compter le nombre de trajets par arrondissement de dépose
dropoff_counts = df_trips['doborough'].value_counts().reset_index()
dropoff_counts.columns = ['Arrondissement', 'Nombre de trajets']
print(dropoff_counts)

# Créer le graphique des arrondissements de dépose
fig = px.bar(dropoff_counts, x='Arrondissement', y='Nombre de trajets', 
             title='Arrondissements de dépose',
             labels={'Arrondissement': 'Arrondissement de dépose', 'Nombre de trajets': 'Nombre de trajets'})

# Personnaliser la mise en page
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10))

# Afficher le graphique
fig.show()

  Arrondissement  Nombre de trajets
0      Manhattan           31870398
1         Queens            1845413
2       Brooklyn            1375151
3          Bronx             207571
4            EWR             103976
5  Staten Island              10038


### Itinéraires les plus populaires entre arrondissements

In [25]:
# Compter le nombre de trajets par paire d'arrondissements
route_counts = df_trips.groupby(['puborough', 'doborough']).size().reset_index(name='Nombre de trajets')
# Créer une nouvelle colonne par concaténation de puborough et doborough
route_counts['Itineraire'] = route_counts.apply(lambda row: f"{row['puborough']}-{row['doborough']}", axis=1)
route_counts = route_counts.sort_values(by='Nombre de trajets', ascending=False)
route_counts

,puborough,doborough,Nombre de trajets,Itineraire
21,Manhattan,Manhattan,29711723,Manhattan-Manhattan
27,Queens,Manhattan,2078477,Queens-Manhattan
22,Manhattan,Queens,1074903,Manhattan-Queens
28,Queens,Queens,749164,Queens-Queens
19,Manhattan,Brooklyn,731743,Manhattan-Brooklyn
25,Queens,Brooklyn,540765,Queens-Brooklyn
18,Manhattan,Bronx,102301,Manhattan-Bronx
7,Brooklyn,Brooklyn,98546,Brooklyn-Brooklyn
20,Manhattan,EWR,97262,Manhattan-EWR
24,Queens,Bronx,77321,Queens-Bronx


In [26]:
# Top 5 des itineraires les plus populaires entre arrondissements
top5_route_counts = route_counts.head()

# Créer le graphique à barres horizontales
fig = px.bar(top5_route_counts, x='Itineraire', y='Nombre de trajets',
             title='Itinéraires les plus populaires entre arrondissements'
             )

# Afficher le graphique
fig.show()

### Zones de prise en charge

In [32]:
# Compter le nombre de trajets par zone de prise en charge
pickup_counts = df_trips['puzone'].value_counts().reset_index()
pickup_counts.columns = ['Zone', 'Nombre de trajets']
pickup_counts = pickup_counts.sort_values(by='Nombre de trajets', ascending=False)
print(pickup_counts)

# Créer le graphique des 20 zones de prise en charge les plus populaires
top20_pickup_counts = pickup_counts.head(20)

fig = px.bar(top20_pickup_counts, x='Nombre de trajets', y='Zone', 
             orientation='h',
             title='Top 20 des Zones de prise en charge les plus populaires',
             labels={'Zone': 'Zone de prise en charge', 'Nombre de trajets': 'Nombre de trajets'})

# Personnaliser la mise en page
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10))

# Afficher le graphique
fig.show()

                                              Zone  Nombre de trajets
0                                      JFK Airport            1849275
1                            Upper East Side South            1712138
2                                   Midtown Center            1685771
3                            Upper East Side North            1507798
4                                     Midtown East            1298008
..                                             ...                ...
254  Governor's Island/Ellis Island/Liberty Island                 10
255              Eltingville/Annadale/Prince's Bay                  9
256                                  West Brighton                  7
257                                  Port Richmond                  6
258                                        Oakwood                  3

[259 rows x 2 columns]


### Zones de deposes

In [34]:
# Compter le nombre de trajets par zone de prise en charge
dropoff_counts = df_trips['dozone'].value_counts().reset_index()
dropoff_counts.columns = ['Zone', 'Nombre de trajets']
dropoff_counts = dropoff_counts.sort_values(by='Nombre de trajets', ascending=False)
print(dropoff_counts)

# Créer le graphique des 20 zones de depose les plus populaires
top20_dropoff_counts = dropoff_counts.head(20)

fig = px.bar(top20_dropoff_counts, x='Nombre de trajets', y='Zone', 
             orientation='h',
             title='Top 20 des Zones de depose les plus populaires',
             labels={'Zone': 'Zone de depose', 'Nombre de trajets': 'Nombre de trajets'})

# Personnaliser la mise en page
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10))

# Afficher le graphique
fig.show()

                                              Zone  Nombre de trajets
0                            Upper East Side North            1595275
1                            Upper East Side South            1530028
2                                   Midtown Center            1405248
3                        Times Sq/Theatre District            1114621
4                                      Murray Hill            1070376
..                                             ...                ...
254                              Rossville/Woodrow                176
255                                    Jamaica Bay                 71
256                                Freshkills Park                 41
257  Governor's Island/Ellis Island/Liberty Island                 17
258                               Great Kills Park                  2

[259 rows x 2 columns]


### Itineraires les plus populaires entre zones

In [37]:
# Compter le nombre de trajets par paire de zones
route_counts_zones = df_trips.groupby(['puzone', 'dozone']).size().reset_index(name='Nombre de trajets')

# Créer une nouvelle colonne par concaténation de puzone et dozone
route_counts_zones['Itinéraire'] = route_counts_zones.apply(lambda row: f"{row['puzone']} TO {row['dozone']}", axis=1)

# Trier par nombre de trajets
route_counts_zones = route_counts_zones.sort_values(by='Nombre de trajets', ascending=False)
print(route_counts_zones)

# Top 20 des itinéraires les plus populaires entre zones
top20_route_counts_zones = route_counts_zones.head(20)

# Créer le graphique à barres horizontales
fig = px.bar(top20_route_counts_zones, x='Itinéraire', y='Nombre de trajets',
             title='Top 20 Itinéraires les plus populaires entre zones'
             )

# Afficher le graphique
fig.show()

                                  puzone                     dozone  \
37122              Upper East Side South      Upper East Side North   
36876              Upper East Side North      Upper East Side South   
37123              Upper East Side South      Upper East Side South   
36875              Upper East Side North      Upper East Side North   
24724                     Midtown Center      Upper East Side South   
...                                  ...                        ...   
30771                      Rockaway Park             Sheepshead Bay   
30774                      Rockaway Park  Springfield Gardens South   
30775                      Rockaway Park              Starrett City   
30776                      Rockaway Park                   Steinway   
11505  East New York/Pennsylvania Avenue                 Whitestone   

       Nombre de trajets                                       Itinéraire  
37122             249331   Upper East Side South TO Upper East Side Nor

## Analyses sur le nombre de passagers

In [11]:
# Définir la taille du lot pour l'itération
chunk_size = 100000

# Lire le fichier parquet et obtenir les métadonnées
parquet_file = pq.ParquetFile('trips_cleaned.parquet')

# Initialiser une liste pour stocker les résultats partiels
partial_results = []

# Itérer sur les segments parquet par petits morceaux
for i in range(0, parquet_file.num_row_groups, chunk_size):
    # Lire un segment parquet
    chunk = parquet_file.read_row_group(i, columns=['puborough', 'doborough', 'passenger_count'])
    # Convertir le segment en DataFrame pandas
    chunk_df = chunk.to_pandas()
    # Créer une nouvelle colonne par concaténation de puborough et doborough pour chaque morceau
    chunk_df['Itineraire Arrondissement'] = chunk_df.apply(lambda row: f"{row['puborough']} - {row['doborough']}", axis=1)
    # Regrouper par Itineraire pour chaque morceau et calculer la moyenne du nombre de passagers
    partial_passengers_per_route = chunk_df.groupby('Itineraire Arrondissement')['passenger_count'].mean().reset_index()
    # Ajouter les résultats partiels à la liste
    partial_results.append(partial_passengers_per_route)

# Concaténer les résultats partiels en un seul DataFrame
passengers_per_route = pd.concat(partial_results)

# Regrouper à nouveau par Itineraire sur l'ensemble des données et calculer la moyenne finale du nombre de passagers
passengers_per_route = passengers_per_route.groupby('Itineraire Arrondissement')['passenger_count'].mean().reset_index()

# Trier les résultats par nombre moyen de passagers de manière décroissante
passengers_per_route = passengers_per_route.sort_values(by='passenger_count', ascending=False)

# Afficher le résultat
passengers_per_route

,Itineraire Arrondissement,passenger_count
15,EWR - Staten Island,4.000000
13,EWR - Manhattan,3.000000
12,EWR - EWR,2.181818
30,Staten Island - EWR,2.000000
18,Manhattan - EWR,1.840620
24,Queens - EWR,1.780405
7,Brooklyn - EWR,1.700000
27,Queens - Staten Island,1.654008
25,Queens - Manhattan,1.549703
20,Manhattan - Queens,1.522834


Ce résultat montre la moyenne du nombre de passagers pour différents itinéraires entre les arrondissements. Voici quelques interprétations possibles :

1. **Itinéraires avec le plus grand nombre moyen de passagers :**
   - "EWR - Staten Island" a la plus grande moyenne avec 4 passagers par trajet.
   - "EWR - Manhattan" suit avec une moyenne de 3 passagers par trajet.

2. **Itinéraires avec une moyenne modérée de passagers :**
   - "EWR - EWR" a une moyenne de 2.18 passagers par trajet.
   - "Staten Island - EWR" a une moyenne de 2 passagers par trajet.

3. **Itinéraires avec une moyenne plus basse de passagers :**
   - Les itinéraires reliant les aéroports aux autres arrondissements ont généralement des moyennes de passagers plus élevées.
   - Les itinéraires entre les arrondissements de Manhattan, Brooklyn et Queens ont des moyennes de passagers variables, allant d'environ 1.4 à 1.8 passagers par trajet.
   - Certains itinéraires, comme "Brooklyn - Brooklyn" et "Bronx - Bronx", ont des moyennes plus basses, autour de 1.0 à 1.3 passagers par trajet.

En résumé, cette analyse fournit des informations sur la densité de passagers pour différents itinéraires entre les arrondissements, ce qui peut être utile pour la planification des transports ou d'autres analyses liées aux déplacements dans la ville.

In [12]:
# Définir la taille du lot pour l'itération
chunk_size = 100000

# Initialiser une liste pour stocker les résultats partiels
partial_results = []

# Itérer sur les segments parquet par petits morceaux
for i in range(0, parquet_file.num_row_groups, chunk_size):
    # Lire un segment parquet
    chunk = parquet_file.read_row_group(i, columns=['puzone', 'dozone', 'passenger_count'])
    # Convertir le segment en DataFrame pandas
    chunk_df = chunk.to_pandas()
    # Créer une nouvelle colonne par concaténation de puzone et dozone pour chaque morceau
    chunk_df['Itineraire Zone'] = chunk_df.apply(lambda row: f"{row['puzone']} - {row['dozone']}", axis=1)
    # Regrouper par Itineraire pour chaque morceau et calculer la moyenne du nombre de passagers
    partial_passengers_per_route = chunk_df.groupby('Itineraire Zone')['passenger_count'].mean().reset_index()
    # Ajouter les résultats partiels à la liste
    partial_results.append(partial_passengers_per_route)

# Concaténer les résultats partiels en un seul DataFrame
passengers_per_route = pd.concat(partial_results)

# Regrouper à nouveau par Itineraire sur l'ensemble des données et calculer la moyenne finale du nombre de passagers
passengers_per_route = passengers_per_route.groupby('Itineraire Zone')['passenger_count'].mean().reset_index()

# Trier les résultats par nombre moyen de passagers de manière décroissante
passengers_per_route = passengers_per_route.sort_values(by='passenger_count', ascending=False)

# Afficher le résultat
passengers_per_route

,Itineraire Zone,passenger_count
12216,Steinway - Saint Albans,6.0
8811,Manhattanville - Greenwich Village North,6.0
8027,Lincoln Square West - Borough Park,6.0
11661,SoHo - Dyker Heights,6.0
8295,Long Island City/Hunters Point - Clinton Hill,6.0
...,...,...
3880,East New York - Kips Bay,1.0
3881,East New York - LaGuardia Airport,1.0
10176,Murray Hill - Crown Heights South,1.0
3882,East New York - Laurelton,1.0


In [14]:
passengers_per_route.head(20)

,Itineraire Zone,passenger_count
8914,Maspeth-Murray Hill,6.0
8915,Maspeth-Parkchester,6.0
9939,Morningside Heights-Flushing,6.0
8811,Manhattanville-Greenwich Village North,6.0
903,Boerum Hill-Borough Park,6.0
2040,Clinton East-Bensonhurst West,6.0
4289,Elmhurst-Stuy Town/Peter Cooper Village,6.0
13001,TriBeCa/Civic Center-Queens Village,6.0
2149,Clinton East-North Corona,6.0
12216,Steinway-Saint Albans,6.0


Dans ces lignes du résultat, nous avons les itinéraires de zone avec le nombre moyen le plus élevé de passagers. Voici une explication pour chacun :

1. **Maspeth-Murray Hill (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Maspeth à Murray Hill et a une moyenne de 6 passagers par trajet en moyenne. Cela indique une forte demande de transport sur cet itinéraire spécifique.

2. **Maspeth-Parkchester (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Maspeth à Parkchester et a également une moyenne de 6 passagers par trajet en moyenne. Il semble y avoir une demande élevée de transport entre ces deux zones.

3. **Morningside Heights-Flushing (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Morningside Heights à Flushing et affiche une moyenne de 6 passagers par trajet en moyenne. Cela suggère une forte activité de déplacement entre ces deux zones.

4. **Manhattanville-Greenwich Village North (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Manhattanville à Greenwich Village North et a également une moyenne de 6 passagers par trajet en moyenne. Cela indique une demande importante de transport entre ces deux zones de Manhattan.

5. **Boerum Hill-Borough Park (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Boerum Hill à Borough Park et a une moyenne de 6 passagers par trajet en moyenne. Cela suggère une forte demande de déplacement entre ces deux zones.

En résumé, ces lignes indiquent les itinéraires de zone avec la plus forte moyenne de passagers par trajet, ce qui peut être utile pour la planification des transports et la gestion des ressources.

In [15]:
passengers_per_route.passenger_count.unique()

array([6.        , 5.        , 4.5       , ..., 1.02777778, 1.02564103,
       1.        ])

In [16]:
passengers_per_route.tail (20)

,Itineraire Zone,passenger_count
3869,East New York-Flatbush/Ditmas Park,1.0
10194,Murray Hill-Erasmus,1.0
3870,East New York-Flatiron,1.0
3871,East New York-Flushing,1.0
3873,East New York-Garment District,1.0
3874,East New York-Gramercy,1.0
3875,East New York-Hammels/Arverne,1.0
3876,East New York-Homecrest,1.0
10185,Murray Hill-East Flushing,1.0
10184,Murray Hill-East Flatbush/Remsen Village,1.0


### Tarif moyen par itineraire de zone

In [20]:
# Définir la taille du lot pour l'itération
chunk_size = 100000

# Lire le fichier parquet et obtenir les métadonnées
parquet_file = pq.ParquetFile('trips_cleaned.parquet')

# Initialiser une liste pour stocker les résultats partiels
partial_results = []

# Itérer sur les segments parquet par petits morceaux
for i in range(0, parquet_file.num_row_groups, chunk_size):
    # Lire un segment parquet
    chunk = parquet_file.read_row_group(i, columns=['puzone', 'dozone', 'total_amount'])
    # Convertir le segment en DataFrame pandas
    chunk_df = chunk.to_pandas()
    # Créer une nouvelle colonne par concaténation de puzone et dozone pour chaque morceau
    chunk_df['Itineraire Zone'] = chunk_df.apply(lambda row: f"{row['puzone']}-{row['dozone']}", axis=1)
    # Calculer le tarif moyen par itinéraire de zone pour chaque morceau
    partial_fare_per_route = chunk_df.groupby('Itineraire Zone')['total_amount'].mean().reset_index()
    # Ajouter les résultats partiels à la liste
    partial_results.append(partial_fare_per_route)

# Concaténer les résultats partiels en un seul DataFrame
fare_per_route = pd.concat(partial_results)

# Regrouper à nouveau par itinéraire de zone sur l'ensemble des données et calculer le tarif moyen final
fare_per_route = fare_per_route.groupby('Itineraire Zone')['total_amount'].mean().reset_index()

# Trier les résultats par tarif moyen de manière décroissante
fare_per_route = fare_per_route.sort_values(by='total_amount', ascending=False)

# Afficher le résultat
fare_per_route

,Itineraire Zone,total_amount
6894,Kensington-Downtown Brooklyn/MetroTech,369.000000
9133,Midtown Center-Bloomfield/Emerson Hill,236.500000
11440,Schuylerville/Edgewater Park-Bellerose,228.000000
12077,Springfield Gardens South-Newark Airport,210.646667
6895,Kensington-East Flatbush/Farragut,201.000000
...,...,...
556,Bay Ridge-Bath Beach,-21.600000
5032,Flushing-Kew Gardens,-23.800000
14956,Williamsburg (North Side)-Crown Heights South,-36.000000
7671,Lenox Hill West-Bensonhurst East,-81.140000


In [17]:
# Calculer le tarif moyen par zone de prise en charge
fare_means = df_trips_with_zones.groupby('zone_dropoff')['total_amount'].mean().reset_index(name='Tarif_moyen')

# Trier les zones par tarif moyen dans l'ordre décroissant
fare_means_sorted = fare_means.sort_values(by='Tarif_moyen', ascending=False)

# Créer un graphique des tarifs moyens par zone de prise en charge en utilisant Plotly Express
fig = px.bar(fare_means_sorted, x='zone_dropoff', y='Tarif_moyen', title='Tarif moyen par zone de depose',
             labels={'zone_dropoff': 'Zone de depose', 'Tarif_moyen': 'Tarif moyen ($)'},
             color='Tarif_moyen', color_continuous_scale='blues')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [18]:
# Convertir les colonnes de date en format datetime
df_trips['tpep_pickup_datetime'] = pd.to_datetime(df_trips['tpep_pickup_datetime'])
df_trips['tpep_dropoff_datetime'] = pd.to_datetime(df_trips['tpep_dropoff_datetime'])

# Calculer la durée du trajet en minutes
df_trips['Trip_duration'] = (df_trips['tpep_dropoff_datetime'] - df_trips['tpep_pickup_datetime']).dt.total_seconds() / 60

# Analyse bivariée avec la durée du trajet
plt.figure(figsize=(10, 6))
plt.scatter(df_trips['Trip_duration'], df_trips['Total_amount'], alpha=0.5)
plt.title('Tarifs de taxi en fonction de la durée du trajet')
plt.xlabel('Durée du trajet (minutes)')
plt.ylabel('Tarif ($)')
plt.show()

: 